In [1]:
!pip install librosa==0.10.1
!!pip install numpy==1.23.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.10.2.post1:
      Successfully uninstalled librosa-0.10.2.post1


In [2]:
# mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# unzip ScreamDataset form mydrive

!unzip /content/drive/MyDrive/ScreamDataset/ScreamDataset.zip -d /content/

Archive:  /content/drive/MyDrive/ScreamDataset/ScreamDataset.zip
  inflating: /content/Converted_Separately/non_scream/1.wav  
  inflating: /content/Converted_Separately/non_scream/10.wav  
  inflating: /content/Converted_Separately/non_scream/100.wav  
  inflating: /content/Converted_Separately/non_scream/1000.wav  
  inflating: /content/Converted_Separately/non_scream/1001.wav  
  inflating: /content/Converted_Separately/non_scream/1002.wav  
  inflating: /content/Converted_Separately/non_scream/1003.wav  
  inflating: /content/Converted_Separately/non_scream/1004.wav  
  inflating: /content/Converted_Separately/non_scream/1005.wav  
  inflating: /content/Converted_Separately/non_scream/1006.wav  
  inflating: /content/Converted_Separately/non_scream/1007.wav  
  inflating: /content/Converted_Separately/non_scream/1008.wav  
  inflating: /content/Converted_Separately/non_scream/1009.wav  
  inflating: /content/Converted_Separately/non_scream/101.wav  
  inflating: /content/Converted_

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Define the path to the dataset
converted_separately_path = '/content/Converted_Separately'

# Paths to the scream and non-scream folders
scream_path = os.path.join(converted_separately_path, 'scream')
non_scream_path = os.path.join(converted_separately_path, 'non_scream')

In [2]:
# length of scream and non_scream folders

# Get the number of files in each directory
scream_files = len([f for f in os.listdir(scream_path) if os.path.isfile(os.path.join(scream_path, f))])
non_scream_files = len([f for f in os.listdir(non_scream_path) if os.path.isfile(os.path.join(non_scream_path, f))])

print("Number of scream files:", scream_files)
print("Number of non-scream files:", non_scream_files)

Number of scream files: 1583
Number of non-scream files: 1545


In [3]:
# Verify that the paths exist
if not os.path.exists(scream_path):
    raise FileNotFoundError(f"The directory {scream_path} does not exist. Please verify the dataset path.")
if not os.path.exists(non_scream_path):
    raise FileNotFoundError(f"The directory {non_scream_path} does not exist. Please verify the dataset path.")

# Function to extract MFCC features from an audio file
def extract_features(file_path, n_mfcc=13):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [4]:
# Prepare the dataset
X = []  # Features
y = []  # Labels

In [5]:
# Process scream files
for file_name in os.listdir(scream_path):
    file_path = os.path.join(scream_path, file_name)
    if os.path.isfile(file_path):  # Ensure it's a file
        features = extract_features(file_path)
        if features is not None:
            X.append(features)
            y.append(1)  # Label for scream

# Process non-scream files
for file_name in os.listdir(non_scream_path):
    file_path = os.path.join(non_scream_path, file_name)
    if os.path.isfile(file_path):  # Ensure it's a file
        features = extract_features(file_path)
        if features is not None:
            X.append(features)
            y.append(0)  # Label for non-scream

In [6]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Train a Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [9]:
# Test the classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [10]:
# Save the model
model_path = 'scream_detector_model.pkl'
joblib.dump(classifier, model_path)

['scream_detector_model.pkl']

In [11]:
# Display results
print(f"Model training completed. Accuracy: {accuracy * 100:.2f}%")
print(f"Model saved at: {model_path}")

Model training completed. Accuracy: 93.45%
Model saved at: scream_detector_model.pkl


In [12]:
import os
import numpy as np
import librosa
import joblib

# Load the saved model
model_path = '/content/scream_detector_model.pkl'
classifier = joblib.load(model_path)

In [13]:
# Function to extract MFCC features from an audio file
def extract_features(file_path, n_mfcc=13):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [14]:
# Function to predict the label of an audio file
def predict_scream(file_path):
    features = extract_features(file_path)
    if features is not None:
        features = np.array([features])  # Reshape for model input
        prediction = classifier.predict(features)
        label = "Scream" if prediction[0] == 1 else "Non-Scream"
        print(f"Prediction: {label}")
        return label
    else:
        print("Could not extract features from the provided file.")
        return None

In [16]:
# User input for file path
file_path = input("Enter the path to the audio file: ")
if os.path.isfile(file_path):
    predict_scream(file_path)
else:
    print("Invalid file path. Please provide a valid audio file.")

Enter the path to the audio file: /content/Converted_Separately/non_scream/141.wav
Prediction: Non-Scream
